# Tutorial2: 图像分类1

可在SCOW HPC集群运行本教程，也可在SCOW AI集群运行。

本节旨在展示使用CNN进行图像分类的简单案例，使用MNIST数据集和一个规模较小的简单CNN网络。

分以下几步来实现：
1. 环境安装
2. 分步运行本文件

    2.1 数据加载和预处理

    2.2 定义 CNN 模型

    2.3 训练与评估

## 1. 环境安装
 
请确保已经执行了 [tutorial_scow_for_ai](../tutorial_scow_for_ai.md) 中的"安装依赖、注册ipykernel"。

建议使用1张910B NPU运行本教程。

## 2. 分步运行本文件

### 2.1 数据预处理

MNIST 是一个手写数字数据集，包含了数字0到9的灰度图像。

~~~bash
# 解压MNIST数据集
cd scow-for-ai-tutorial-ascend/Tutorial2_classification/
tar -xzvf data.tar.gz
~~~

In [ ]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets

# 定义数据转换方式: 标准化
transform = transforms.Compose([
    transforms.ToTensor(),  # 将图像转换为 PyTorch 张量
    transforms.Normalize((0.5,), (0.5,))  # 标准化（均值0.5，标准差0.5）
])

# 加载训练集和测试集
train_dataset = datasets.MNIST('./data', train=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, transform=transform)


## 2.2 定义 CNN 模型

In [ ]:

import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 卷积层：输入1通道，输出32通道，卷积核3x3，填充1
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        # 池化层：用于减小输出矩阵大小
        self.mp = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.mp(self.conv1(x)))
        x = self.relu(self.mp(self.conv2(x)))
        x = x.view(x.size(0), -1)  # 展平
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## 2.3 训练与评估

完成模型训练和评估在 910B 加速卡上需要约 15 min

In [ ]:
from torch.utils.data import DataLoader
import torch
import torch_npu
import time
from datetime import timedelta

# 参数设置
learning_rate, epochs, batch_size = 0.001, 3, 64
if torch.npu.is_available():
    device = torch.device('npu:0')
else:
    device = torch.device('cpu')
print('device:', device)

# 实例化模型
model = CNN().to(device)

# 损失函数
criterion = nn.CrossEntropyLoss()

# 优化器
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

# 评估函数
def accuracy(model, data_loader, device):
    total = 0
    correct = 0
    model.eval()
    for X, y in data_loader:
        X, y = X.to(device), y.to(device)
        outputs = model(X)
        _, predicted = torch.max(outputs.data, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()

    return 100 * correct / total
    
# 数据准备
train_loader = DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=False)

# 训练模型
train_ls = []
train_acc, test_acc = [], []
for epoch in range(epochs):
    start_time = time.time()
    model.train() 
    total_loss = 0
    
    for X, y in train_loader:
        X, y = X.to(device), y.to(device)
        
        # 前向传播
        outputs = model(X)
        loss = criterion(outputs, y)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # loss 记录
        total_loss += loss.item()
    average_loss = total_loss / len(train_loader)
    train_ls.append(average_loss)
        
    # 模型评估
    with torch.no_grad():
        train_acc.append(accuracy(model, train_loader, device))
        test_acc.append(accuracy(model, test_loader, device))
    
    # 计时
    end_time = time.time()
    print(f"{epoch}/{epochs}: accuracy: {test_acc[-1]}")
    print(f"time consuming: {timedelta(seconds=end_time - start_time)}")

训练过程中 Loss 一直在下降，本次训练中最高能达到 99% 的准确率，实际上通过改进模型和训练过程并进行数据增强，CNN训练MNIST数据集的准确率可以进一步提升。

In [ ]:
import matplotlib.pyplot as plt
from matplotlib_inline import backend_inline

backend_inline.set_matplotlib_formats('svg')

plt.rcParams['figure.figsize'] = (4, 3)

fig, ax1 = plt.subplots()

# loss
ax1.plot(list(range(1, epochs + 1)), train_ls, 'b-', label='train Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss', color='b')
ax1.tick_params(axis='y', labelcolor='b')
ax1.set_yscale('log')
ax1.set_xlim([1, epochs])

# Accuracy
ax2 = ax1.twinx()  
ax2.plot(list(range(1, epochs + 1)),
         train_acc, 'r-', label='train accuracy')
ax2.plot(list(range(1, epochs + 1)),
         test_acc, 'r--', label='test accuracy')
ax2.set_ylabel('Accuracy (%)', color='r')
ax2.tick_params(axis='y', labelcolor='r')

fig.legend()

---

> 作者: 黎颖; 褚苙扬; 龙汀汀
>
> 联系方式: yingliclaire@pku.edu.cn; cly2412307718@stu.pku.edu.cn; l.tingting@pku.edu.cn